In [9]:
%reload_ext autoreload
%autoreload 2

## Read .cle file

In [66]:
import pandas as pd
import pathlib
from datetime import datetime, timedelta

year = 2015

fi = pathlib.Path(f'/home/gfogwil/Documentos/Trabajo/instrumentos/dmps/data/processed/events-{year}.csv')
fo = pathlib.Path(f'/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/dmps/events_{year}.csv')

events = pd.read_csv(fi,
                     names=['file', 'start', 'end', 'type'])

labels = pd.DataFrame()


for idx, line in events.iterrows():
#     start = datetime.strptime(f"{line['file'][2:]} {line['start']}", '%Y%m%d %H:%M:%S')
#     end = datetime.strptime(f"{line['file'][2:]} {line['end']}", '%Y%m%d %H:%M:%S')
    start = datetime.strptime(f"{line['file']} {line['start']}", '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(f"{line['file']} {line['end']}", '%Y-%m-%d %H:%M:%S')

    if line['type'] == 'ne':
        label = pd.DataFrame(index=pd.date_range(start.date(), periods=144, freq='10Min'))
        label['label'] = 'ne'
        labels = labels.append(label)
    else:
        try:
            labels[start.date()]
            labels.loc[start:end, 'label'] = 'e'
            
        except KeyError:
            label = pd.DataFrame(index=pd.date_range(start.date(), periods=144, freq='10Min'))
            label['label'] = 'ne'

            if line['type'] == 'e':
                label.loc[start:end, 'label'] = 'e'
            labels = labels.append(label)
    labels = labels[~labels.index.duplicated(keep='last')]

In [67]:
labels.sort_index().to_csv(fo)

## Copio los archivos de datos

In [46]:
import shutil
events = pd.read_csv(f'/home/gfogwil/Documentos/Trabajo/DMPS/events_{year}.csv',
                     names=['file', 'start', 'end', 'type'])

labels = pd.DataFrame()

inv_path = pathlib.Path('/home/gfogwil/Documentos/Trabajo/DMPS/wetransfer-73815c/inverted/')
out_path = pathlib.Path('/home/gfogwil/Documentos/Trabajo/DMPS/cherrypicked_events/')
for idx, line in events.iterrows():
    line['file'] = line['file'].lower()
    src = inv_path / f"{line['file'][:4]}/dm{line['file'].replace('-','')}.sum"
    dst = out_path / f"dm{line['file'].replace('-','')}.cle"
#     print(dst)
    shutil.copyfile(src, dst)

In [22]:
import shutil
events = pd.read_csv(f'/home/gfogwil/Documentos/Trabajo/DMPS/events_{year}.csv',
                     names=['file', 'start', 'end', 'type'])

In [33]:
events.iloc[2]['file'].replace('-','')

'20160210'